In [7]:
# imports 

import numpy as np
import heapq
import math
import scipy as sp
from matplotlib import pyplot

In [ ]:
#################### TODO ####################

# write a load function that uses the approach outlined in carpark
# must handle over capacity 

# write a scheduling function that augments the paths of elevators

# We can copy and paste the main code and change scheduling/loading algorithm 

In [3]:
#################### PARAMETERS ####################

# m         : the number of floors in the building
# n         : the number of elevators 
# lambdas   : m * m array; i,j is the rate of arrivals on floor i who want to go to floor j
#             lambda are function with time parameter
# occupancy : max occupancy of an elevator
# h         : length of simulation

#################### SIMULATION DESIGN ####################

# Elevators have predetermined paths
# On button press, scheduler decides which paths change
# When elevator reaches a floor people get off then get on

# Clocks for changes in button states
#        and next stop on elevator path

#################### NOTES ####################

# buttons indicate first arrival of type down/up
# therefore the amount of people that go down/up
# at elevator arrival is that minus time of first arrival

# floors are labeled 0 ... m - 1
# assume time to travel between floors is constant

#################### DATA STRUCTURES ####################

# Feel free to modify, but provide documentation

# down  : tuple where index j is rate of clients of floor j going down
# total : tuple where index j is sum of all rates

# elevators: list of elevator structures

# elevator structure: clients    : list where index j is amount of people want to go to floor j
#                     path       : lists of pairs (floor number, time of arrival to floor)
#                     floor      : the current floor

# floors: list of floor structures

# floor structure: clients : list where index j is amount of people want to go to floor j
#                  button  : list of pairs (time of press, type of press)



def sim(m: int, n: int, lambdas: tuple[tuple[float]], occupancy: int, h: int):
    # Initialization 
    rng       = np.random.default_rng() # rv generator
    t         = 0                       # total elapsed time
    floors    = list( [] for _ in range(m) )
    elevators = list( [np.zeros(m), [], 0] for _ in range(n) )
    down      = tuple( sum(lambdas[j][:j]) for j in range(m) )
    total     = tuple( sum(lambdas[j]) for j in range(m) )
    clocks    = []
    # Initialize clocks/floors
    for i in range(m):
        pair = [rng.exponential(1/total[i])] # time of press
        # type of press
        if rng.uniform(0, 1) <= down[i] / total[i]:
            pair.append(-1)
        else:
            pair.append(1)
        clocks.append(pair)
        floors[i].append(pair)
    heapq.heapify(clocks)

    while(t <= h):
        # SCHEDULER SECTION
        time, dir = heapq.heappop(clocks)

        # ELEVATOR SECTION


    return # performance metrics

In [4]:
#tracking the clocks of button change of every floor
def DES_engine(m: int, lambda_param, h: int, buckets):
    rng = np.random.default_rng() # rv generator
    t = 0.0
    n = 0
    generate_tuples = lambda m: [(0, 0) for _ in range(m)]
    generate_clocks = lambda m: [rng.exponential(1/lambda_param) for _ in range(m)]
    floor_states = generate_tuples(m)
    button_arrival = generate_clocks(m)
    print(generate_tuples(5))
    print(generate_clocks(5))
    q_up = 0
    q_down = 0
    
    SOF = [button_arrival, floor_states, q_up, q_down]
    
    while (t<h):
        next = min(button_arrival)
        tau = next[0]
        floor = button_arrival.index(tau)
        for i in buckets:
            if i < up:
                